# Azure AI агенти са подршком за Model Context Protocol (MCP)

Овај нотебук демонстрира како користити Azure AI агенте са алатима за Model Context Protocol (MCP) како би се креирао интелигентни агент који може користити спољне MCP сервере за побољшане могућности.


## Инсталирајте неопходне NuGet пакете

Прво, потребно је да инсталирамо Azure AI Agents Persistent пакет који обезбеђује основну функционалност за рад са Azure AI Agents.


## Предности аутентификације без кључа

Овај нотебук демонстрира **аутентификацију без кључа**, која пружа неколико предности:
- ✅ **Нема потребе за управљањем API кључевима** - Користи аутентификацију засновану на Azure идентитету
- ✅ **Побољшана безбедност** - Нема чувања тајни у коду или конфигурацији
- ✅ **Аутоматска ротација акредитива** - Azure управља животним циклусом акредитива
- ✅ **Приступ заснован на улогама** - Користи Azure RBAC за детаљну контролу дозвола

`DefaultAzureCredential` ће аутоматски користити најбољи доступни извор акредитива:
1. Управљани идентитет (приликом покретања у Azure-у)
2. Акредитиви Azure CLI (током развоја)
3. Акредитиви Visual Studio-а
4. Променљиве окружења (ако су конфигурисане)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Инсталирајте Azure Identity пакет за аутентификацију са Azure услугама користећи DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Увоз потребних именских простора

Увезите потребне именске просторе за Azure AI Agents и Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Конфигурисање Azure AI Agent Client (Аутентикација без кључа)

Подесите променљиве за конфигурацију и креирајте PersistentAgentsClient користећи **аутентикацију без кључа**:
- **projectEndpoint**: Крајња тачка Azure AI Foundry пројекта
- **modelDeploymentName**: Назив распоређеног AI модела (GPT-4.1 nano)
- **mcpServerUrl**: URL MCP сервера (Microsoft Learn API)
- **mcpServerLabel**: Ознака за идентификацију MCP сервера
- **DefaultAzureCredential**: Користи управљани идентитет, Azure CLI или друге изворе акредитива (нису потребни API кључеви)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Креирај MCP дефиницију алата

Креирај MCP дефиницију алата која се повезује са Microsoft Learn MCP сервером. Ово ће омогућити агенту приступ садржају и документацији на Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Направите AI агента

Направите трајног AI агента са одређеним моделом и MCP алатима. Агенат је конфигурисан са:
- GPT-4.1 nano моделом
- Упутствима за коришћење MCP алата за помоћ
- Приступом Microsoft Learn MCP серверу


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Креирајте нит и пошаљите поруку

Креирајте нит разговора и пошаљите корисничку поруку са питањем о разлици између Azure OpenAI и OpenAI. Ово ће тестирати способност агента да користи MCP алате за пружање тачних информација.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Конфигуришите MCP алатке ресурсе (Без кључа)

Подесите ресурсе за MCP алатку. За потпуно приступ без кључа, можете уклонити прилагођене хедере ако MCP сервер подржава аутентификацију засновану на Azure идентитету. Пример испод показује како да додате хедере ако је потребно, али за сценарије без кључа, ови можда неће бити неопходни.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Покрени рад агента

Креирај и покрени процес за обраду поруке корисника. Агент ће користити конфигурисане MCP алате и ресурсе за генерисање одговора.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Праћење рада агента и одобравање алата (без кључа)

Пратите статус рада агента и управљајте одобравањем потребних алата. Овај циклус:
1. Чека да се рад заврши или да захтева акцију
2. Аутоматски одобрава позиве MCP алата када је то потребно
3. За аутентификацију без кључа, заглавља можда нису потребна ако MCP сервер подржава Azure идентификацију


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Приказ резултата разговора

Прегледајте и прикажите све поруке у нитима, укључујући и корисниково питање и одговор агента. Поруке се приказују хронолошким редом са временским ознакама и индикаторима улога.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
